# Find Optimal Parameters

Subsequently, find optimal parameters.

In the first place, record average ranking for each parameters.
Larger ticks are tend to be more influence the result of trading, therefore, calculate the ranking ordered by bottom -> middle -> top.
Finally regard these optimal paramaters as the representative paramaters.

続いてはこの中から最適なパラメータを探索する。

まず各timeframeごとに、それぞれのパラメーターに対して平均的な順位を記録する。
結果への影響力は、短期的な価格変動の効果を受けにくい順に大きいため、bottom -> middle -> topの順で平均的な順位を計算する。
その中で得られた最適なパラメータを各タイムフレームの代表的なパラメーターとする。

In [1]:
%matplotlib inline
import research_kit_bottom_trend_follow as rk

merged_results = rk.get_joined_params_and_summary()

normal_results = merged_results.query("inverse_trading == False")
inversed_results = merged_results.query("inverse_trading == True")

downloading BTC/USD data on bitmex
downloading 2020-01-06 22:52:30 ~ 2020-01-07 07:12:30 data
50.0% completed
downloading 2020-01-07 07:12:30 ~ 2020-01-07 11:09:18.317032 data
100.0% completed
Loading OHLCV data from bitmex_original_ohlcv_1min now...
data loaded


In [4]:
def find_optimal_parameters_by_ranking(df, verbose=False):
    sorted_df_in_need = df[[
        "timeframe",
        "bottom_trend_tick",
        "middle_trend_tick",
        "top_trend_tick",
        "profit_factor"]
    ].sort_values("profit_factor", ascending=False)
    
    timeframe_params = [60,30,10,1]
    bottom_trend_tick_params = [5,10,15,20,25,30,35,40,45,50]
    middle_trend_tick_params = [3,5,7,9,12,15,18,21,24,27,30]
    top_trend_tick_params = [1,2,3,4,5,6,7,8]
    
    for timeframe in timeframe_params:
        print("calc optimal parameters for timeframe:" + str(timeframe))
        sorted_df_in_need_with_timeframe = sorted_df_in_need.query("timeframe ==" + str(timeframe))
        if verbose:
            print("bottom")
        optimal_bottom = find_best_ranking_parameter(sorted_df_in_need_with_timeframe,
            bottom_trend_tick_params, verbose=verbose)
        if verbose:
            print("middle with bottom => " + str(optimal_bottom) )
        optimal_middle = find_best_ranking_parameter(sorted_df_in_need_with_timeframe,
            middle_trend_tick_params, bottom_trend_tick=optimal_bottom, verbose=verbose)
        if verbose:
            print("top with bottom => " + str(optimal_bottom) + " middle => " + str(optimal_middle))
        optimal_top = find_best_ranking_parameter(sorted_df_in_need_with_timeframe,
            top_trend_tick_params, bottom_trend_tick=optimal_bottom, middle_trend_tick=optimal_middle, verbose=verbose)
        
        print("optimal bottom => " + str(optimal_bottom) + " middle => " +\
              str(optimal_middle) + "top => " + str(optimal_top))

In [8]:
find_optimal_parameters_by_ranking(normal_results, verbose=False)
find_optimal_parameters_by_ranking(inversed_results, verbose=False)

calc optimal parameters for timeframe:60
optimal bottom => 5 middle => 3top => 2
calc optimal parameters for timeframe:30
optimal bottom => 10 middle => 3top => 1
calc optimal parameters for timeframe:10
optimal bottom => 5 middle => 3top => 1
calc optimal parameters for timeframe:1
optimal bottom => 10 middle => 3top => 1
calc optimal parameters for timeframe:60
optimal bottom => 5 middle => 3top => 1
calc optimal parameters for timeframe:30
optimal bottom => 10 middle => 3top => 1
calc optimal parameters for timeframe:10
optimal bottom => 5 middle => 3top => 1
calc optimal parameters for timeframe:1
optimal bottom => 10 middle => 3top => 1


# Find Optimal Parameters Summary

The result shows that bottom=>5, middle=>3, top=>1 is the best combination for all timeframe except for timeframe=60.
However, timeframe=60 is no more important than shorter timeframe, so it's ignorable.
From now, discuss with expected optimal hyperparamaters timeframe=>1, bottom=>5, middle=>3, top=>1 and focus on details under timeframe=1.

timeframe=60を除いたすべての時間軸に対してbottom=>5, middle=>3, top=>1が最適であるという結果が得られた。
timeframe=60は、より短い時間足に比べてそこまで重要ではないので無視する。
ここまでで時間足が1, bottomが5, middleが3, topが1の場合が最適であることが予想される。
ここからはまず、この時間足について詳細に検討する。